# Tree Methods Consulting Project 

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.
___

**Think carefully about what this problem is really asking you to solve. While we will use Machine Learning to solve this, it won't be with your typical train/test split workflow. If this confuses you, skip ahead to the solution code along walk-through!**
____

# Good Luck!

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Dog Food Tree Method Consulting").getOrCreate()

In [3]:
data = spark.read.csv("dog_food.csv", inferSchema=True, header=True)

In [4]:
data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [6]:
data.show()

+---+---+----+---+-------+
|  A|  B|   C|  D|Spoiled|
+---+---+----+---+-------+
|  4|  2|12.0|  3|    1.0|
|  5|  6|12.0|  7|    1.0|
|  6|  2|13.0|  6|    1.0|
|  4|  2|12.0|  1|    1.0|
|  4|  2|12.0|  3|    1.0|
| 10|  3|13.0|  9|    1.0|
|  8|  5|14.0|  5|    1.0|
|  5|  8|12.0|  8|    1.0|
|  6|  5|12.0|  9|    1.0|
|  3|  3|12.0|  1|    1.0|
|  9|  8|11.0|  3|    1.0|
|  1| 10|12.0|  3|    1.0|
|  1|  5|13.0| 10|    1.0|
|  2| 10|12.0|  6|    1.0|
|  1| 10|11.0|  4|    1.0|
|  5|  3|12.0|  2|    1.0|
|  4|  9|11.0|  8|    1.0|
|  5|  1|11.0|  1|    1.0|
|  4|  9|12.0| 10|    1.0|
|  5|  8|10.0|  9|    1.0|
+---+---+----+---+-------+
only showing top 20 rows



In [7]:
data.describe().show()

+-------+------------------+------------------+------------------+------------------+-------------------+
|summary|                 A|                 B|                 C|                 D|            Spoiled|
+-------+------------------+------------------+------------------+------------------+-------------------+
|  count|               490|               490|               490|               490|                490|
|   mean|  5.53469387755102| 5.504081632653061| 9.126530612244897| 5.579591836734694| 0.2857142857142857|
| stddev|2.9515204234399057|2.8537966089662063|2.0555451971054275|2.8548369309982857|0.45221563164613465|
|    min|                 1|                 1|               5.0|                 1|                0.0|
|    max|                10|                10|              14.0|                10|                1.0|
+-------+------------------+------------------+------------------+------------------+-------------------+



Vector Assembler 

In [8]:
from pyspark.ml.feature import VectorAssembler

In [9]:
data.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [10]:
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'], outputCol='features')

In [12]:
#transform data using assembler
output = assembler.transform(data)

Random Forest Calssification

In [13]:
from pyspark.ml.classification import RandomForestClassifier

In [14]:
rfc = RandomForestClassifier(featuresCol="features", labelCol="Spoiled")

In [15]:
output.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)
 |-- features: vector (nullable = true)



In [16]:
final_data = output.select('Spoiled', 'features')

In [17]:
final_data.show()

+-------+-------------------+
|Spoiled|           features|
+-------+-------------------+
|    1.0| [4.0,2.0,12.0,3.0]|
|    1.0| [5.0,6.0,12.0,7.0]|
|    1.0| [6.0,2.0,13.0,6.0]|
|    1.0| [4.0,2.0,12.0,1.0]|
|    1.0| [4.0,2.0,12.0,3.0]|
|    1.0|[10.0,3.0,13.0,9.0]|
|    1.0| [8.0,5.0,14.0,5.0]|
|    1.0| [5.0,8.0,12.0,8.0]|
|    1.0| [6.0,5.0,12.0,9.0]|
|    1.0| [3.0,3.0,12.0,1.0]|
|    1.0| [9.0,8.0,11.0,3.0]|
|    1.0|[1.0,10.0,12.0,3.0]|
|    1.0|[1.0,5.0,13.0,10.0]|
|    1.0|[2.0,10.0,12.0,6.0]|
|    1.0|[1.0,10.0,11.0,4.0]|
|    1.0| [5.0,3.0,12.0,2.0]|
|    1.0| [4.0,9.0,11.0,8.0]|
|    1.0| [5.0,1.0,11.0,1.0]|
|    1.0|[4.0,9.0,12.0,10.0]|
|    1.0| [5.0,8.0,10.0,9.0]|
+-------+-------------------+
only showing top 20 rows



In [18]:
#model out data
rfc_model = rfc.fit(final_data)

In [20]:
final_data.head(1)

[Row(Spoiled=1.0, features=DenseVector([4.0, 2.0, 12.0, 3.0]))]

In [19]:
rfc_model.featureImportances

SparseVector(4, {0: 0.0175, 1: 0.0207, 2: 0.9464, 3: 0.0153})

Now FeatureImportances function helped us understand that Preservative C is nearly 1 which makes it more important feature in this analysis which may 
be causing the early spolage of Dog food!!. 